## Data cleaning of movie data set

### Trying to reduce data size of 18000++ 

#### We will be doing this by dropping movies without release_date, budget and revenue(if both are missing, since it becomes inaccurate to fill in 2 unknown values) since they do not provide any meaning without these values. We will also drop movie whose release data are before 1927 and after 2022 as we are not concerned with those time periods since oscar starts for movies from 1927 till present.

#### The assumption is that movies without both budget and revenue will not be able to provide much value to our dataset

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from sklearn.impute import SimpleImputer

Importing csv data and viewing first 5 rows


In [2]:
data = pd.read_csv("oscar_nom_extract.csv")
# find the column with missing id
print(data[data['id'].isnull()])
data = data.dropna(subset=['id']).reset_index(drop=True)
# convert column id from float to int
data['id'] = data['id'].astype(int)
data.head()

      id title  budget  revenue  release_date  popularity  vote_average  \
4562 NaN   NaN     NaN      NaN           NaN         NaN           NaN   

      runtime  top_casts_popularity_avg  casts_popularity_sum  \
4562      NaN                       NaN                   NaN   

      top_cast_popularity  top_crews_popularity_avg  crews_popularity_sum  \
4562                  NaN                       NaN                   NaN   

      top_crew_popularity  
4562                  NaN  


,id,title,budget,revenue,release_date,popularity,vote_average,runtime,top_casts_popularity_avg,casts_popularity_sum,top_cast_popularity,top_crews_popularity_avg,crews_popularity_sum,top_crew_popularity
0,113167,The Noose,NaN,NaN,1928.0,0.600,NaN,65.0,1.582286,15.598,2.260,0.722000,5.654,0.980
1,43197,The Last Command,NaN,NaN,1955.0,2.108,6.40,110.0,11.207714,109.128,21.641,1.352143,15.417,2.167
2,104212,A Ship Comes In,NaN,NaN,1928.0,0.790,5.50,70.0,1.531000,11.317,4.116,1.016571,7.716,1.380
3,82474,7th Heaven,119.0,NaN,1927.0,5.695,7.37,110.0,2.758571,25.023,8.574,1.284000,20.293,2.268
4,42538,Sadie Thompson,1000000.0,7000000.0,1928.0,1.643,6.80,97.0,2.442286,18.896,6.750,1.789000,13.723,2.301


In [3]:
data.info()
data.shape[0]
totallength = len(data)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5655 entries, 0 to 5654
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        5655 non-null   int32  
 1   title                     5655 non-null   object 
 2   budget                    1868 non-null   float64
 3   revenue                   1905 non-null   float64
 4   release_date              5640 non-null   float64
 5   popularity                5655 non-null   float64
 6   vote_average              5316 non-null   float64
 7   runtime                   5612 non-null   float64
 8   top_casts_popularity_avg  5101 non-null   float64
 9   casts_popularity_sum      5101 non-null   float64
 10  top_cast_popularity       5101 non-null   float64
 11  top_crews_popularity_avg  5588 non-null   float64
 12  crews_popularity_sum      5588 non-null   float64
 13  top_crew_popularity       5588 non-null   float64
dtypes: float

There are 5655 rows

Restting index in case the indexing is not in order

In [4]:
data.reset_index(drop=True)

,id,title,budget,revenue,release_date,popularity,vote_average,runtime,top_casts_popularity_avg,casts_popularity_sum,top_cast_popularity,top_crews_popularity_avg,crews_popularity_sum,top_crew_popularity
0,113167,The Noose,NaN,NaN,1928.0,0.600,NaN,65.0,1.582286,15.598,2.260,0.722000,5.654,0.980
1,43197,The Last Command,NaN,NaN,1955.0,2.108,6.400,110.0,11.207714,109.128,21.641,1.352143,15.417,2.167
2,104212,A Ship Comes In,NaN,NaN,1928.0,0.790,5.500,70.0,1.531000,11.317,4.116,1.016571,7.716,1.380
3,82474,7th Heaven,119.0,NaN,1927.0,5.695,7.370,110.0,2.758571,25.023,8.574,1.284000,20.293,2.268
4,42538,Sadie Thompson,1000000.0,7000000.0,1928.0,1.643,6.800,97.0,2.442286,18.896,6.750,1.789000,13.723,2.301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5650,913743,A House Made of Splinters,NaN,NaN,2023.0,5.041,7.000,87.0,NaN,NaN,NaN,1.198286,15.588,1.400
5651,926993,Haulout,NaN,NaN,2022.0,2.104,7.261,25.0,0.600000,0.600,0.600,0.708571,8.560,0.980
5652,785398,EO,NaN,NaN,2022.0,287.612,6.547,88.0,5.975143,52.406,15.522,2.637571,49.697,3.927
5653,916405,The Quiet Girl,NaN,1756887.0,2022.0,15.584,7.600,94.0,2.463714,25.812,4.756,1.304571,14.026,1.932


Dropping rows where both "budget" and "revenue" are both null values since it would be difficult to fill in values using strategies like median when we do not know how inflation will affect these variables

In [15]:
#for i in tqdm(range(totallength)):
#    data.dropna(subset=["budget","revenue"],how="all",inplace=True,axis =0)

100%|██████████| 5655/5655 [00:02<00:00, 2326.37it/s]


The data now has 9983 entries

In [5]:
print(data.shape)
print(data["release_date"].isnull().value_counts())

(5655, 14)
False    5640
True       15
Name: release_date, dtype: int64


From the values, it looks like there are no movies with missing release_date

Excluding movies whose release data are before 1927 and after 2022 since first oscar was for movies released at 1927 and latest oscar is for movies released at 2022

In [10]:
data = pd.DataFrame(data[(data["release_date"]<=2022) & (data["release_date"]>=1927)])
data.shape
totallength = len(data)

In [11]:
data2 = data.copy()
data2 = data2.reset_index(drop=True)

obtaining percentage revenue based on budget (revenue/budget)*100% to compare percentage rather than aboslute values to account for inflation

In [12]:
percentage_revenue =[]
for k in range(0,totallength):
    if(math.isnan(data2.iloc[k].at["revenue"]) or math.isnan(data2.iloc[k].at["budget"])):
        percentage_revenue.append(np.nan)
    else:
        percent = round(((data2.iloc[k].at["revenue"])/(data2.iloc[k].at["budget"])*100),2)
        percentage_revenue.append(percent)

In [13]:
print(len(percentage_revenue))
p_revenue={}
p_revenue["percentage_revenue"]=percentage_revenue
percent_revenue = pd.DataFrame(p_revenue)
print(percent_revenue)
data3 = pd.concat([data2,percent_revenue],axis=1)


5621
      percentage_revenue
0                    NaN
1                    NaN
2                    NaN
3                    NaN
4                  700.0
...                  ...
5616                 NaN
5617                 NaN
5618                 NaN
5619                 NaN
5620                 NaN

[5621 rows x 1 columns]


Importing csv file which contains all best picture winning movies from 1927 to 2022

In [14]:
data3

,id,title,budget,revenue,release_date,popularity,vote_average,runtime,top_casts_popularity_avg,casts_popularity_sum,top_cast_popularity,top_crews_popularity_avg,crews_popularity_sum,top_crew_popularity,percentage_revenue
0,113167,The Noose,NaN,NaN,1928.0,0.600,NaN,65.0,1.582286,15.598,2.260,0.722000,5.654,0.980,NaN
1,43197,The Last Command,NaN,NaN,1955.0,2.108,6.400,110.0,11.207714,109.128,21.641,1.352143,15.417,2.167,NaN
2,104212,A Ship Comes In,NaN,NaN,1928.0,0.790,5.500,70.0,1.531000,11.317,4.116,1.016571,7.716,1.380,NaN
3,82474,7th Heaven,119.0,NaN,1927.0,5.695,7.370,110.0,2.758571,25.023,8.574,1.284000,20.293,2.268,NaN
4,42538,Sadie Thompson,1000000.0,7000000.0,1928.0,1.643,6.800,97.0,2.442286,18.896,6.750,1.789000,13.723,2.301,700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5616,685691,"BARDO, False Chronicle of a Handful of Truths",NaN,NaN,2022.0,18.330,7.058,160.0,4.778143,55.332,9.362,6.080143,62.381,7.378,NaN
5617,926993,Haulout,NaN,NaN,2022.0,2.104,7.261,25.0,0.600000,0.600,0.600,0.708571,8.560,0.980,NaN
5618,785398,EO,NaN,NaN,2022.0,287.612,6.547,88.0,5.975143,52.406,15.522,2.637571,49.697,3.927,NaN
5619,916405,The Quiet Girl,NaN,1756887.0,2022.0,15.584,7.600,94.0,2.463714,25.812,4.756,1.304571,14.026,1.932,NaN


In [15]:
winners = pd.read_csv("winner_nom.csv")

In [16]:
winners.head()
# name refers to the company who won

,Unnamed: 0,year_film,film,winner
0,0,1927,The Noose,False
1,1,1927,The Last Command,True
2,2,1927,A Ship Comes In,False
3,3,1927,7th Heaven,True
4,4,1927,Sadie Thompson,False


dropping Index, year_ceremony(the year when oscar ceremony took place),ceremony, category since they are all best pictures and name of company who won

In [19]:
winners = winners[['film', 'winner']]
winners.shape

(5098, 2)

Removing any duplicate movies

In [20]:
data4 = data3.drop_duplicates(subset=["title"],keep='first')
data4 = data4.reset_index(drop=True)


In [21]:
print(winners.shape) #keeping track of the dimensions
print(data4.shape) 
print(winners.head())
print(data4.head())

(5098, 2)
(4922, 15)
               film  winner
0         The Noose   False
1  The Last Command    True
2   A Ship Comes In   False
3        7th Heaven    True
4    Sadie Thompson   False
       id             title     budget    revenue  release_date  popularity  \
0  113167         The Noose        NaN        NaN        1928.0       0.600   
1   43197  The Last Command        NaN        NaN        1955.0       2.108   
2  104212   A Ship Comes In        NaN        NaN        1928.0       0.790   
3   82474        7th Heaven      119.0        NaN        1927.0       5.695   
4   42538    Sadie Thompson  1000000.0  7000000.0        1928.0       1.643   

   vote_average  runtime  top_casts_popularity_avg  casts_popularity_sum  \
0           NaN     65.0                  1.582286                15.598   
1          6.40    110.0                 11.207714               109.128   
2          5.50     70.0                  1.531000                11.317   
3          7.37    110.0        

Combining the 2 data together

In [22]:
name_list = []
index_list = []
Win = []
for a in range(0,5098): #obtain names of award winning films
    name_list.append(winners.iloc[a].at["film"])

for b in range(0,4922): #obtain index of award winning films
    if data4.iloc[b].at["title"] in name_list:
        index_list.append(b)

print(len(index_list)) #ensure there are 95

for c in range(0,4922): #add true for index in index_list, false otherwise
    if c in index_list:
        Win.append(True)
    else:
        Win.append(False)

Win_dictionary = {}
Win_dictionary["Won"]= Win
Win_df = pd.DataFrame(Win_dictionary)
data5 = pd.concat([data4,Win_df],axis=1)

4459


In [23]:
#checking the contents
print(data5.head())
print(data5.shape)

       id             title     budget    revenue  release_date  popularity  \
0  113167         The Noose        NaN        NaN        1928.0       0.600   
1   43197  The Last Command        NaN        NaN        1955.0       2.108   
2  104212   A Ship Comes In        NaN        NaN        1928.0       0.790   
3   82474        7th Heaven      119.0        NaN        1927.0       5.695   
4   42538    Sadie Thompson  1000000.0  7000000.0        1928.0       1.643   

   vote_average  runtime  top_casts_popularity_avg  casts_popularity_sum  \
0           NaN     65.0                  1.582286                15.598   
1          6.40    110.0                 11.207714               109.128   
2          5.50     70.0                  1.531000                11.317   
3          7.37    110.0                  2.758571                25.023   
4          6.80     97.0                  2.442286                18.896   

   top_cast_popularity  top_crews_popularity_avg  crews_popularity_s

Filling NAN or null values using SimpleImputer and median.
median is chosen as any missing values will be a numerical variable
"budget" and "revenue" has been replaced by "percetage_revenue"

Removing first "popularity"
Both "popularity" and "popualarity.1" refers to the same thing it's just the data was taken on different days

In [24]:
data5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4922 entries, 0 to 4921
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        4922 non-null   int32  
 1   title                     4922 non-null   object 
 2   budget                    1679 non-null   float64
 3   revenue                   1701 non-null   float64
 4   release_date              4922 non-null   float64
 5   popularity                4922 non-null   float64
 6   vote_average              4633 non-null   float64
 7   runtime                   4894 non-null   float64
 8   top_casts_popularity_avg  4450 non-null   float64
 9   casts_popularity_sum      4450 non-null   float64
 10  top_cast_popularity       4450 non-null   float64
 11  top_crews_popularity_avg  4870 non-null   float64
 12  crews_popularity_sum      4870 non-null   float64
 13  top_crew_popularity       4870 non-null   float64
 14  percenta

In [25]:
data6=data5.drop(["popularity"],axis=1)
print(data6.head())

       id             title     budget    revenue  release_date  vote_average  \
0  113167         The Noose        NaN        NaN        1928.0           NaN   
1   43197  The Last Command        NaN        NaN        1955.0          6.40   
2  104212   A Ship Comes In        NaN        NaN        1928.0          5.50   
3   82474        7th Heaven      119.0        NaN        1927.0          7.37   
4   42538    Sadie Thompson  1000000.0  7000000.0        1928.0          6.80   

   runtime  top_casts_popularity_avg  casts_popularity_sum  \
0     65.0                  1.582286                15.598   
1    110.0                 11.207714               109.128   
2     70.0                  1.531000                11.317   
3    110.0                  2.758571                25.023   
4     97.0                  2.442286                18.896   

   top_cast_popularity  top_crews_popularity_avg  crews_popularity_sum  \
0                2.260                  0.722000                 5

In [26]:
impute = SimpleImputer(missing_values=np.nan,strategy="median")
imputer= impute.fit(data6.iloc[:,5:15])
data6.iloc[:,5:15]=imputer.transform(data6.iloc[:,5:15])

Export a cleaned movie data csv

In [27]:
data6.to_csv("CleanedNomData.csv")